In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import os
import math

In [2]:
# Set model name here, options = {'iMM904', 'iYO844', 'iML1515'}
model_name = 'iYO844'

# Set path to the original datasets and saving directory
RAW_DATA_DIR = f'../raw/{model_name}/'
SAVE_DIR = f'./{model_name}/'

## Load Data and Get Node Feature
---

### 1. Reaction data - `Model.csv`

In [3]:
model = pd.read_csv(f'{RAW_DATA_DIR}Model.csv')

print(model.shape)
# Drop useless columns
model = model.loc[:, ['Rule', 'reac_meta_name', 'reac_meta_value', 'Gibbs', 'kcat', 'km']]
model.head()

(1521, 15)


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN


### 2. Molecule data - `mvec.csv`

In [4]:
mvec = pd.read_csv(f'{RAW_DATA_DIR}mvec.csv')
if model_name == 'iML1515':
    mvec.rename(columns={'meta_name': 'reac_meta_name'}, inplace=True)
    
print(mvec.shape)
mvec = mvec.drop(columns=['SMILES'])
mvec.head()

(619, 130)


,reac_meta_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,istnt_c,0.034616,0.052344,-0.032405,0.053052,-0.000082,-0.009388,-0.003238,-0.019588,0.006186,...,-0.007310,-0.001641,0.032719,-0.041696,0.001186,-0.010220,-0.033718,0.046453,0.007658,0.020313
1,hmbil_c,0.064757,0.106059,-0.085903,0.128846,0.018197,-0.014582,-0.001312,-0.049914,0.016280,...,-0.026731,0.008829,0.069371,-0.088491,0.002651,-0.033185,-0.076106,0.087244,0.015458,0.031300
2,tag6p__D_c,0.078863,0.114675,-0.105692,0.158943,0.023062,-0.020216,0.002675,-0.065022,0.022953,...,-0.025510,0.011606,0.070363,-0.117186,-0.005045,-0.047840,-0.076804,0.101711,0.004547,0.045300
3,fruur_c,0.068635,0.094918,-0.089442,0.126414,0.008797,-0.012519,0.002537,-0.058893,0.017126,...,-0.021080,0.003307,0.066820,-0.096962,-0.007166,-0.044819,-0.071273,0.095626,0.008698,0.032080
4,g3pg_c,0.066607,0.091234,-0.090395,0.127275,0.014320,-0.011735,0.003785,-0.055166,0.028846,...,-0.017062,0.004865,0.060276,-0.098263,-0.004310,-0.029819,-0.063236,0.089087,0.005903,0.026193


### 3. Enzyme feature - `padded_features.pkl`

Return:

- Logits of all enzymes:
    - logits:(1600*1600*2)
- Single representations:
    - `e_single_dict = {Rule: (1600,)}`

In [5]:
e_raw_feature = pickle.load(open((f'{RAW_DATA_DIR}padded_features.pkl'), 'rb'))
e_single_dict = dict()

# Save the single representations into e_single_dict
for e in e_raw_feature:
    e_single_dict[e] = e_raw_feature[e]['single']

### 4. Molecule feature - combine the reaction data and molecule data

```
m_feature_dict = {reac_meta_name: (129,)}
```

In [6]:
m_feature_dict = dict()

# extract the Gibbs info of each molecule from reaction data
Gibbs = model.loc[:, ['reac_meta_name', 'Gibbs']].drop_duplicates(
    subset='reac_meta_name', keep='first')
Gibbs.set_index('reac_meta_name', inplace=True)

## convert Gibbs into float
Gibbs['Gibbs'] = Gibbs['Gibbs'].apply(lambda x: re.sub("[^-.0-9]", "", str(x)))
Gibbs['Gibbs'] = Gibbs['Gibbs'].apply(lambda x: float(x) if x != '' else 0.0)


def convert_feature_to_list(x):
    reac_meta_name = x['reac_meta_name']
    gibbs_value = [float(Gibbs.loc[reac_meta_name, 'Gibbs'])]
    return np.array(gibbs_value + list(x['0':'127']))


mvec['feature'] = mvec.apply(convert_feature_to_list, axis=1)

m_feature_dict_t = mvec.loc[:, ['reac_meta_name', 'feature']].set_index('reac_meta_name').to_dict('index')

for k in m_feature_dict_t:
    m_feature_dict[k] = m_feature_dict_t[k]['feature']
    

print(len(m_feature_dict))
print('Dimension of the molecule feature: {}'.format(len(m_feature_dict[list(m_feature_dict.keys())[0]])))


619
Dimension of the molecule feature: 129


## Data Clean for Construct the Reaction Graph
---

Enzyme:

- Some enzymes in model do not have af2 output
- i.e. delete the rows in model that refer to enzyme without key in enzyme_feature_dict

Molecule:

- Some molecules in model do not have mvec infomation
- i.e. delete the rows in model that refer to molecules without info in mvec


In [7]:
# Check missing enzymes in af2 output
e_model = list(model['Rule'].unique())      # enzymes in model sheet
e_af2 = list(e_single_dict.keys())    # enzymes in af2 output

enzymes = pd.DataFrame({'model': e_model}, columns=['model'])
enzymes['in_af2'] = enzymes['model'].apply(lambda x: x in e_af2)

no_af2_output = enzymes[enzymes['in_af2'] == False].model
print('Number of enzyme without af2 output = {}'.format(len(no_af2_output)))
print(list(no_af2_output))


# Delete the rows in model that refer to enzyme without af2 output
print('> Delete the rows in model that refer to enzyme without af2 output')

print('Before deleting, there are {} links in model.'.format(model.shape[0]))
model_new = model.loc[:]
model_new['in_af2_output'] = model_new['Rule'].apply(
    lambda x: x in e_single_dict.keys())
model_new = model_new[model_new['in_af2_output'] == True]
print('After deleting, there are {} links in model.'.format(
    model_new.shape[0]))
model_new.head()


Number of enzyme without af2 output = 2
['BSU02480', 'BSU31980']
> Delete the rows in model that refer to enzyme without af2 output
Before deleting, there are 1521 links in model.
After deleting, there are 1515 links in model.


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km,in_af2_output
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243,True
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN,True
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN,True
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN,True
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN,True


In [8]:
# Delete the rows in model that refer to molecules without mvec infomation
print('> Delete the rows in model that refer to molecules without mvec infomation')

print('Before deleting, numer of links in model: {}'.format(
    model_new.shape[0]))
model_moles = model_new.reac_meta_name.unique()     # molecules in model sheet
mvec_moles = mvec.reac_meta_name.unique()           # molecules in mvec

model_new['mol_in_mvec'] = model_new['reac_meta_name'].apply(
    lambda x: x in list(mvec_moles))
model_new = model_new[model_new['mol_in_mvec'] == True]
print('After deleting, numer of links in model: {}'.format(model_new.shape[0]))
model_new.head()

> Delete the rows in model that refer to molecules without mvec infomation
Before deleting, numer of links in model: 1515
After deleting, numer of links in model: 1458


,Rule,reac_meta_name,reac_meta_value,Gibbs,kcat,km,in_af2_output,mol_in_mvec
0,BSU00090,imp_c,-1.0,-244.30493,NaN,0.0243,True,True
1,BSU00090,xmp_c,1.0,-291.6249,NaN,NaN,True,True
2,BSU00140,dcmp_c,1.0,-206.3048,NaN,NaN,True,True
3,BSU00140,dcyt_c,-1.0,17.714563,NaN,NaN,True,True
4,BSU00140,adn_c,-1.0,87.03457,NaN,NaN,True,True


## Graph Neural Network Data

---

### Node

In [9]:
# Enzyme

# node_id, node_name, node_type_id(0)
num_e = model_new['Rule'].unique().shape[0]
print('Number of nodes of node_type_id 0: {}'.format(num_e))

# node_id, node_name, node_type_id
node_e = pd.DataFrame({'node_id': np.arange(num_e), 
                        'node_name': model_new['Rule'].unique(), 
                        'node_type_id': np.zeros_like(model_new['Rule'].unique())},
                      columns=['node_id', 'node_name', 'node_type_id'])

# node_feature
node_e['node_feature'] = node_e['node_name'].apply(lambda x: e_single_dict[x])

node_e

Number of nodes of node_type_id 0: 384


,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"[-2.6357093, 11.089254, 0.9175112, -2.2136374,..."
1,1,BSU00140,0,"[14.018574, 11.753677, 13.034294, 11.752765, 1..."
2,2,BSU00150,0,"[15.961157, 12.483061, 6.694125, 2.3762176, -1..."
3,3,BSU00180,0,"[10.051447, 2.882035, 6.5983353, -8.677919, 2...."
4,4,BSU00270,0,"[12.062081, 6.2870865, -1.5195656, -1.3795595,..."
...,...,...,...,...
379,379,BSU40190,0,"[11.876185, 10.247246, 11.630852, 10.800863, 1..."
380,380,BSU40320,0,"[-2.6511145, 1.0494355, -0.45718634, 4.1283865..."
381,381,BSU40340,0,"[17.341208, 9.80417, 8.620768, 13.817736, 7.69..."
382,382,BSU40420,0,"[2.231642, 7.7314253, 8.377593, 4.8179655, 8.0..."


In [10]:
# Get a dictionary where keys are the node_name and the values are node_id

e_id_dict = {item[0]:item[1]['node_id'] for item in (node_e.loc[:, ['node_name', 'node_id']].set_index('node_name').to_dict('index')).items()}
e_id_dict

print(len(e_id_dict))

384


In [11]:
# Save the logits tensor of each enzyme to {SAVE_DIR}logits/
counter = 0
for rule in e_id_dict:
    rule_logits = e_raw_feature[rule]['logits']
    with open(f'{SAVE_DIR}logits/{e_id_dict[rule]}.npy', 'wb') as f:
        np.save(f, rule_logits)
    counter += 1

print(counter)

384


In [12]:
# Molecule

num_m = model_new['reac_meta_name'].unique().shape[0]
print('Number of nodes of node_type_id 1: {}'.format(num_m))

# node_id, node_name, node_type_id(1)
node_m = pd.DataFrame({'node_id': np.arange(num_e, num_e+num_m),
                       'node_name': model_new['reac_meta_name'].unique(),
                       'node_type_id': np.ones_like(model_new['reac_meta_name'].unique())},
                      columns=['node_id', 'node_name', 'node_type_id'])

# mol_feature
node_m['node_feature'] = node_m['node_name'].apply(lambda x: m_feature_dict[x])

node_m

Number of nodes of node_type_id 1: 616


,node_id,node_name,node_type_id,node_feature
0,384,imp_c,1,"[-244.30493, 0.043830927, 0.07581868, -0.05839..."
1,385,xmp_c,1,"[-291.6249, 0.05696255, 0.089833654, -0.069855..."
2,386,dcmp_c,1,"[-206.3048, 0.047462117, 0.070710495, -0.04960..."
3,387,dcyt_c,1,"[17.714563, 0.044684604, 0.07995401, -0.070259..."
4,388,adn_c,1,"[87.03457, 0.038609814, 0.05554623, -0.0555064..."
...,...,...,...,...
611,995,msa_c,1,"[-81.3787, 0.026036164, 0.046005037, -0.039838..."
612,996,2pcpgc_c,1,"[-162.07196, 0.058406197, 0.10162693, -0.07970..."
613,997,quc_c,1,"[-101.98197, 0.059084572, 0.100022756, -0.0800..."
614,998,glcn__D_c,1,"[-157.02974, 0.04241948, 0.055048183, -0.05639..."


In [13]:
# Concat enzyme and molecule data

node = pd.concat((node_e.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']],
                  node_m.loc[:, ['node_id', 'node_name', 'node_type_id', 'node_feature']]), axis=0)

print('Number of nodes: {}'.format(node.shape[0]))
node.head()

Number of nodes: 1000


,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"[-2.6357093, 11.089254, 0.9175112, -2.2136374,..."
1,1,BSU00140,0,"[14.018574, 11.753677, 13.034294, 11.752765, 1..."
2,2,BSU00150,0,"[15.961157, 12.483061, 6.694125, 2.3762176, -1..."
3,3,BSU00180,0,"[10.051447, 2.882035, 6.5983353, -8.677919, 2...."
4,4,BSU00270,0,"[12.062081, 6.2870865, -1.5195656, -1.3795595,..."


In [14]:
# Save

node.to_pickle(f'{SAVE_DIR}node.pkl')
print(f'Node info saved to {SAVE_DIR}node.pkl')

node = pickle.load(open(f'{SAVE_DIR}node.pkl', 'rb'))
node.head()

Node info saved to ./iYO844/node.pkl


,node_id,node_name,node_type_id,node_feature
0,0,BSU00090,0,"[-2.6357093, 11.089254, 0.9175112, -2.2136374,..."
1,1,BSU00140,0,"[14.018574, 11.753677, 13.034294, 11.752765, 1..."
2,2,BSU00150,0,"[15.961157, 12.483061, 6.694125, 2.3762176, -1..."
3,3,BSU00180,0,"[10.051447, 2.882035, 6.5983353, -8.677919, 2...."
4,4,BSU00270,0,"[12.062081, 6.2870865, -1.5195656, -1.3795595,..."


In [15]:
# Get a dictionary where keys are the node_name and the values are node_id

id_dict = {item[0]:item[1]['node_id'] for item in (node.loc[:, ['node_name', 'node_id']].set_index('node_name').to_dict('index')).items()}
id_dict

print(len(id_dict))

1000


### Link

In [16]:
# Get reaction relations
link = model_new.loc[:, ['Rule', 'reac_meta_name', 'reac_meta_value']]
link.columns = ['enzyme_name', 'mole_name', 'reac_value']

# Get the index of the enzyme and molecule
link['e_idx'] = link['enzyme_name'].apply(lambda x: id_dict[x])
link['m_idx'] = link['mole_name'].apply(lambda x: id_dict[x])

# edge_type_id
link['edge_type_id'] = link['reac_value'].apply(lambda x: 1 if x > 0 else 0)

# node_id_source
link['node_id_source'] = link.apply(
    lambda x: x['e_idx'] if x['edge_type_id'] == 1 else x['m_idx'], axis=1)

# node_id_target
link['node_id_target'] = link.apply(
    lambda x: x['m_idx'] if x['edge_type_id'] == 1 else x['e_idx'], axis=1)

# edge_weight
link['edge_weight'] = link['reac_value'].apply(lambda x: abs(x))

# Reorganize the dataframe
link = link.loc[:, ['node_id_source',
                    'node_id_target', 'edge_type_id', 'edge_weight']]
link.sort_values(by=['edge_type_id', 'node_id_source'], inplace=True)

print('Number of edges: {}'.format(link.shape[0]))
link.head()

Number of edges: 1458


,node_id_source,node_id_target,edge_type_id,edge_weight
0,384,0,0,1.0
208,384,60,0,1.0
1517,384,382,0,1.0
184,385,54,0,1.0
256,385,64,0,1.0


In [17]:
# Save

# link.to_pickle(f'{SAVE_DIR}link.pkl')
# print(f'Link info saved to {SAVE_DIR}link.pkl')

# link = pickle.load(open(f'{SAVE_DIR}link.pkl', 'rb'))
# link.head()

link.to_csv(f'{SAVE_DIR}link.dat',
            sep='\t', index=False, header=False)
print(f'Link info saved to {SAVE_DIR}link.dat')

Link info saved to ./iYO844/link.dat


### Label

In [18]:
def get_combined_label(x, target='km'):
    km, kcat = 0.0, 0.0
    
    if target == 'km':
        km = math.log(x['km']+1)
    elif target == 'kcat':
        kcat = math.log(x['kcat']+1)

    return km#[km, kcat]


def check_nan(x, target='km'):
    if target == 'km':
        return (not pd.isnull(x['km']))
    elif target == 'kcat':
        return (not pd.isnull(x['kcat']))
    
    
def cal_mean_label(df):
    l = list(df['node_label'])
    arr = np.array(l)
    return np.mean(arr, axis=0)
    

def get_link_df(model_new, target='km'):
    label_temp = model_new.loc[:, ['Rule', 'km', 'kcat']]
    label_temp.columns = ['node_name', 'km', 'kcat']

    # drop rows without target information
    label_temp = label_temp[label_temp.apply(lambda x: check_nan(x, target), axis=1)]

    # node_id
    label_temp['node_id'] = label_temp['node_name'].apply(lambda x: id_dict[x])
    
    # node_label
    label_temp['node_label'] = label_temp.apply(lambda x: get_combined_label(x, target), axis=1)
    
    # get the mean of the node_label
    node_labels = label_temp.groupby(['node_id']).apply(cal_mean_label)
    label = pd.DataFrame(node_labels)
    label.columns = ['node_label']
    label.reset_index(inplace=True)
    
    # node_type_id
    label['node_type_id'] = np.zeros(label.shape[0], dtype=int)
    
    label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
    
    return label

label = get_link_df(model_new, 'km')
print('Number of nodes with label: {}'.format(label.shape[0]))
label.head()

Number of nodes with label: 60


,node_id,node_type_id,node_label
0,0,0,0.024009
1,31,0,3.433987
2,33,0,0.246860
3,38,0,2.799109
4,47,0,3.221871


In [19]:
# Save

from sklearn.model_selection import train_test_split

label_train, label_test = train_test_split(label, test_size=0.3)

def save_sing_label(label, file_name):
    label.to_csv('{}/{}'.format(SAVE_DIR, file_name), sep='\t', index=False,
                 header=False, columns=['node_id', 'node_type_id', 'node_label'])
    label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
    print('{} saved to ./{}/{}'.format(file_name, model_name, file_name))
    print('Samples number: {}'.format(label.shape[0]))
    

def save_dual_label(label, file_name):
    label['node_label_to_save'] = label['node_label'].apply(
        lambda x: str(x).split('[')[1].split(']')[0])
    label.to_csv('{}/{}'.format(SAVE_DIR, file_name), sep='\t', index=False,
                 header=False, columns=['node_id', 'node_type_id', 'node_label_to_save'])
    label = label.loc[:, ['node_id', 'node_type_id', 'node_label']]
    print('{} saved to ./{}/{}'.format(file_name, model_name, file_name))
    print('Samples number: {}'.format(label.shape[0]))
    
# save_label(label_mean, 'label')
save_sing_label(label_train, 'label.dat')
save_sing_label(label_test, 'label.dat.test')

label.dat saved to ./iYO844/label.dat
Samples number: 42
label.dat.test saved to ./iYO844/label.dat.test
Samples number: 18
